In [0]:
import os
#!kaggle competitions download -c titanic

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')
x,y = train.drop(['Survived'],axis=1),train.Survived

In [0]:
def preprocess(x):
        
    x = x.drop(['PassengerId','Ticket'],axis=1)

    x.Embarked = pd.Categorical(x.Embarked)
    x.Embarked = x.Embarked.cat.codes

    x['FamilySize'] = x.Parch + x.SibSp
    
    x['isAlone'] = x.FamilySize.map(lambda p: 1 if (p==1) else 0)
    
    x.Cabin = x.Cabin.isna()
    x.Name = x.Name.str.extract('.+ (.+)\.')
    x.Name = pd.Categorical(x.Name)
    x.Name = x.Name.cat.codes

    x.Sex = pd.Categorical(x.Sex)
    x.Sex = x.Sex.cat.codes
    return x

x = preprocess(x)
test = preprocess(test_csv)

In [0]:
x_train,y_train = x,y
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.20,shuffle=True,random_state=42)
#x_train = pd.get_dummies(x_train, columns=['Name','Embarked','Sex'])
#x_val = pd.get_dummies(x_val, columns=['Name','Embarked','Sex'])
age_fill_value = x_train.Age.mean()
fare_fill_value = x_train.Fare.mean()

In [0]:
x_train['Age'] = x_train.Age.fillna(age_fill_value)
x_val['Age'] = x_val.Age.fillna(age_fill_value)
test['Age'] = test.Age.fillna(age_fill_value)
test['Fare'] = test.Fare.fillna(fare_fill_value)

In [0]:
'''
missing_cols = set( x_train.columns ) - set( x_val.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    x_val[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
x_val = x_val[x_train.columns]
'''

In [0]:
x_train['Age_Pclass'] = x_train.Age * x_train.Pclass
test['Age_Pclass'] = test.Age * test.Pclass
x_val['Age_Pclass'] = x_val.Age * x_val.Pclass

x_train['FarePerPerson'] = x_train.Fare/(x_train.FamilySize+1)
x_val['FarePerPerson'] = x_val.Fare/(x_val.FamilySize+1)
test['FarePerPerson'] = test.Fare/(test.FamilySize+1)
from sklearn.ensemble import RandomForestClassifier
reg = RandomForestClassifier(n_jobs=-1,n_estimators=1000,min_samples_leaf=5,max_features ='log2',random_state=42,oob_score=True)
reg.fit(x_train,y_train)

In [18]:
import math

print(reg.score(x_train,y_train))
print(reg.score(x_val,y_val))

0.8848314606741573
0.8324022346368715


In [0]:
for i in test.columns:
    print(i,'-',test[i].isna().sum())

In [0]:
prediction = reg.predict(test)
deep_output = pd.DataFrame({'PassengerId':test_csv['PassengerId'],
                            'Survived':prediction})
deep_output.to_csv('output.csv',index=False)